# Which data records we will look at?

Since our task is to create recommendation system for **users**, suggesting **products** to them,
it is reasonable to think, that we should take user's and product's data into account.

---

### User's data
Data associated with users is directly stored in **users.jsonl** file. 
<br><br>
Record of the user consists of:
- user id in the system
- username
- user city of residence
- user address (which is named street)

Presumably, not all information is useful for us. Further analysis will determine, which features will be discarded.

---

### Product's data

Data associated with products is directly stored in **products.jsonl** file.
<br><br>
Record of the product consists of:
- product id in the system
- product name
- product category path
- price of the product
- rating of the product

Presumably, not all information is useful for us. Further analysis will determine, which features will be discarded.

---


### Session's data
Data connecting users with products is stored in **sessions.jsonl** file.
This file contains records of user's activities within our website.
This activities consist of product viewing or buying.
<br><br>
Record of the session consists of:
- session id in the system
- timestamp of the activity event
- id of the user involved in the event
- id of the product involved in the event
- event type
- offered discount information
- id of the purchase

In order to obtain full information about sessions, we have to
include **products.jsonl** and **users.jsonl** file in our analysis.
<br>
Presumably, not all information is useful for us. Further analysis will determine, which features will be discarded.

---

### Irrelevant data
Data which won't be used during our analysis is stored in file **deliveries.jsonl**. This file contains information about deliveries of products. Content of the file is unrelated to the subject of our work.
We also consider data on place of residence (street) to be unnecessary.

    

In [2]:
import pandas as pd

# Used for verbose data presentation.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

usersDataPath = './data/raw/users.jsonl'
productsDataPath = './data/raw/products.jsonl'
sessionsDataPath = './data/raw/sessions.jsonl'

usersDF = pd.read_json(usersDataPath, lines=True)
sessionsDF = pd.read_json(sessionsDataPath, lines=True)
productsDF = pd.read_json(productsDataPath, lines=True)

# Missing data records
First step in our analysis is to check whether our data contains entries with missing values (null and nan). 

In [3]:
dataWithNames = [(usersDF,    'users'), 
                 (productsDF, 'products'), 
                 (sessionsDF, 'sessions')]

for dataEntry in dataWithNames:
    print('{} table info:'.format(dataEntry[1]))
    dataEntry[0].info()
    print('\n\n')

users table info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  200 non-null    int64 
 1   name     200 non-null    object
 2   city     200 non-null    object
 3   street   200 non-null    object
dtypes: int64(1), object(3)
memory usage: 6.4+ KB



products table info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319 entries, 0 to 318
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     319 non-null    int64  
 1   product_name   319 non-null    object 
 2   category_path  319 non-null    object 
 3   price          319 non-null    float64
 4   user_rating    319 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 12.6+ KB



sessions table info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125146 entries, 0 to 125145
Data 

As we can see in above's cell, there are no missing values in user and products data. In sessions data there are missing values for **purchuse_id**. We believe that this fact is correlated with **event_type** column. We perform further analysis to check if the correlation takes place. 

In [5]:
sessionsDF.loc[sessionsDF['purchase_id'].isna()]['event_type'].value_counts()

AttributeError: 'DataFrameGroupBy' object has no attribute 'value_counts'

Conclusion: session has missing **purchase_id** if it's **event_type** value is equal to **VIEW_PRODUCT**. <br>
Rest of the data has **no missing values**.

## Values and distibutions

Now we will take a closer look at the data values and how they are distributed.

### Products data

Below we present some product records.

In [5]:
productsDF.head(n=10)

,product_id,product_name,category_path,price,user_rating
0,1001,Telefon Siemens Gigaset DA310,Telefony i akcesoria;Telefony stacjonarne,58.97,4.740862
1,1002,Kyocera FS-1135MFP,Komputery;Drukarki i skanery;Biurowe urządzenia wielofunkcyjne,2048.50,1.564504
2,1003,Kyocera FS-3640MFP,Komputery;Drukarki i skanery;Biurowe urządzenia wielofunkcyjne,7639.00,3.520694
3,1004,Fallout 3 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99,4.334193
4,1005,Szalone Króliki Na żywo i w kolorze (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99,4.496124
5,1006,Call of Duty 4 Modern Warfare (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,59.90,0.405119
6,1007,Dead Space 3 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,89.99,1.889997
7,1008,Tom Clancy&#39;s Rainbow Six Vegas (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99,2.097467
8,1009,Kinect Joy Ride (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,69.00,0.304420
9,1010,BioShock 2 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,89.99,3.894822


Just from the "head" of the data we can see that products vary a lot. For example, the price of the 3rd product is many times bigger than for the 1st one. <br><br>
The **category_path** column looks interesting to us. We are curious, how many categories and how many products in each category can be found.

In [6]:
print(productsDF['category_path'].drop_duplicates().reset_index(drop=True).sort_values().to_string(index=False))

                                    Gry i konsole;Gry komputerowe
                    Gry i konsole;Gry na konsole;Gry PlayStation3
                        Gry i konsole;Gry na konsole;Gry Xbox 360
   Komputery;Drukarki i skanery;Biurowe urządzenia wielofunkcyjne
                                  Komputery;Monitory;Monitory LCD
                            Komputery;Tablety i akcesoria;Tablety
                                       Sprzęt RTV;Audio;Słuchawki
         Sprzęt RTV;Przenośne audio i video;Odtwarzacze mp3 i mp4
                                 Sprzęt RTV;Video;Odtwarzacze DVD
               Sprzęt RTV;Video;Telewizory i akcesoria;Anteny RTV
               Sprzęt RTV;Video;Telewizory i akcesoria;Okulary 3D
Telefony i akcesoria;Akcesoria telefoniczne;Zestawy głośnomówiące
  Telefony i akcesoria;Akcesoria telefoniczne;Zestawy słuchawkowe
                          Telefony i akcesoria;Telefony komórkowe
                        Telefony i akcesoria;Telefony stacjonarne


As we can see, categories form a hierarchy. There is a root of each category family, which has sub-families associated with it.

In [7]:
productsDF.groupby('category_path')['product_id'].count().sort_values(ascending=False)

category_path
Gry i konsole;Gry komputerowe                                        202
Gry i konsole;Gry na konsole;Gry Xbox 360                             32
Sprzęt RTV;Video;Telewizory i akcesoria;Anteny RTV                    30
Komputery;Monitory;Monitory LCD                                       17
Gry i konsole;Gry na konsole;Gry PlayStation3                          9
Komputery;Drukarki i skanery;Biurowe urządzenia wielofunkcyjne         9
Telefony i akcesoria;Akcesoria telefoniczne;Zestawy głośnomówiące      5
Telefony i akcesoria;Akcesoria telefoniczne;Zestawy słuchawkowe        4
Komputery;Tablety i akcesoria;Tablety                                  2
Sprzęt RTV;Przenośne audio i video;Odtwarzacze mp3 i mp4               2
Sprzęt RTV;Video;Odtwarzacze DVD                                       2
Telefony i akcesoria;Telefony komórkowe                                2
Sprzęt RTV;Audio;Słuchawki                                             1
Sprzęt RTV;Video;Telewizory i akcesor

Above result show us, that data is not distributed evenly. Some categories have a lot of data when others have hardly any. Luckily, categories form a hierarchy. The solution to uneven distribution problem may to squeeze some categories into single one. <br><br>
By squeezing the data, we could obtain following results:
- "Gry komputerowe": 202
- "Gry na konsole": 41 (Xbox and PS3 Games)
- "Sprzęt RTV": 36
- "Komputery": 26
- "Telefony i akcesoria": 12

These categories and their cardinality seem to be a reasonable by now. <br><br>

Now we will take a closer look at the **product_name** and **product_id** column. We are searching for duplicates to check, whether the data is not corrupted. 

In [8]:
print(productsDF['product_id'].unique().shape[0] == productsDF.shape[0])
print(productsDF['product_name'].unique().shape[0] == productsDF.shape[0])

True
True


As we see, all names and ids are distinct. <br><br>

Now we take a look at the **price** and **user_rating** columns. 

In [9]:
productsDF['price'].describe()

count     319.000000
mean      247.787962
std       813.788548
min         1.000000
25%        24.990000
50%        41.000000
75%        92.995000
max      7639.000000
Name: price, dtype: float64

Let's see how the price is distributed regarding the **category_path** column

In [10]:
productsDF.groupby('category_path')['price'].describe().sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
category_path,,,,,,,,
Komputery;Drukarki i skanery;Biurowe urządzenia wielofunkcyjne,9.0,4238.504444,1872.985317,1998.14,2399.0000,4598.000,5259.0000,7639.00
Komputery;Tablety i akcesoria;Tablety,2.0,2066.995000,353.588746,1816.97,1941.9825,2066.995,2192.0075,2317.02
Telefony i akcesoria;Telefony komórkowe,2.0,1203.615000,1408.959759,207.33,705.4725,1203.615,1701.7575,2199.90
Komputery;Monitory;Monitory LCD,17.0,955.805294,693.800589,269.00,609.0000,739.000,1079.0000,3029.00
Sprzęt RTV;Audio;Słuchawki,1.0,553.000000,NaN,553.00,553.0000,553.000,553.0000,553.00
Telefony i akcesoria;Akcesoria telefoniczne;Zestawy słuchawkowe,4.0,405.735000,607.475026,14.99,44.9900,151.995,512.7400,1303.96
Telefony i akcesoria;Akcesoria telefoniczne;Zestawy głośnomówiące,5.0,217.350000,105.525740,86.28,129.0000,245.000,299.0000,327.47
Sprzęt RTV;Video;Odtwarzacze DVD,2.0,151.000000,59.396970,109.00,130.0000,151.000,172.0000,193.00
Sprzęt RTV;Video;Telewizory i akcesoria;Anteny RTV,30.0,110.775667,47.243374,29.99,79.9900,106.000,133.5000,219.00


We won't comment single example classes. <br><br>

We see that the most 3 expensive classes are connected with the electronic devices. These classes are "sparse" classes. <br> 
The cheapest class is the largest one.

In [11]:
productsDF['user_rating'].describe()

count    319.000000
mean       2.648154
std        1.488174
min        0.013905
25%        1.342497
50%        2.780512
75%        3.932414
max        4.993596
Name: user_rating, dtype: float64

Worth notice is that all values are plausible, that means in dataset provided we don't find any anomalies. That being said, there are three products with suspiciously low prices - __1.0__.
We are going to check is that values are correct by asking our lecturer, as he said that provided that may not be final.

In [12]:
productsDF.loc[productsDF["price"] == productsDF["price"].min()]

,product_id,product_name,category_path,price,user_rating
140,1141,Król Futbolu Piłkarski Quiz (PC),Gry i konsole;Gry komputerowe,1.0,3.462897
192,1193,Heroes Over Europe (PC),Gry i konsole;Gry komputerowe,1.0,4.549431
271,1272,The Ball (PC),Gry i konsole;Gry komputerowe,1.0,2.286441


# Merged data analysis

In this section we will focus on the combined data analysis

## Merged data missing values
The next question, which comes to our mind, is how the data "looks" together. We want to answer questions such as:
- Is there a user, who is not present in any session?
- Is there a product, which is not present in any session?
- Is there a session with unknown user or product?

To answer these questions, we have to look at the joined data.

In [13]:
# Use outer join in order to keep all values from data frames.
mergedDF = pd.merge(sessionsDF, usersDF, how='outer', on='user_id')
mergedDF = pd.merge(mergedDF, productsDF, how='outer', on='product_id')

mergedDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125146 entries, 0 to 125145
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   session_id        125146 non-null  int64         
 1   timestamp         125146 non-null  datetime64[ns]
 2   user_id           125146 non-null  int64         
 3   product_id        125146 non-null  int64         
 4   event_type        125146 non-null  object        
 5   offered_discount  125146 non-null  int64         
 6   purchase_id       10893 non-null   float64       
 7   name              125146 non-null  object        
 8   city              125146 non-null  object        
 9   street            125146 non-null  object        
 10  product_name      125146 non-null  object        
 11  category_path     125146 non-null  object        
 12  price             125146 non-null  float64       
 13  user_rating       125146 non-null  float64       
dtypes: d

As we see, answer to all above's questions is negative. 

One key element is to verify that the coverage of the received data is good enough for the prediction task. To check this we created a heat map with columns being offered products and rows being individual users.
If user had any interaction with product (either viewed it or bought) there is __1__ in corresponding field. Otherwise filed will be empty.

In [18]:
df = sessionsDF.drop(columns=["session_id", "timestamp", "event_type", "offered_discount", "purchase_id"])
df["hit"] = 1
heatMapDF = pd.pivot_table(df, index="user_id", columns="product_id", values="hit")
print("Sparse matrix density: " + str((heatMapDF.size - heatMapDF.isna().sum().sum())/heatMapDF.size))


Sparse matrix density: 0.47782131661442007


As wee can see the density of heat map is ~0,48% which is quiet good for our task.